In [1]:
from pybit.unified_trading import HTTP

import dotenv
import os

from time import sleep, time
import datetime as dt

import pandas as pd

import tqdm

dotenv.load_dotenv('secrets.env')

API_KEY = os.getenv('API')
SECRET_KEY = os.getenv('SECRET')

session = HTTP(
    testnet=False,
    api_key=API_KEY,
    api_secret=SECRET_KEY,
)

In [2]:
base = int(dt.datetime(2022, 1, 1, 0, 00, 00, 000000).timestamp() * 1000)
dt.datetime.fromtimestamp(base / 1000).strftime('%Y-%m-%d %H:%M:%S')

'2022-01-01 00:00:00'

In [14]:
60 * 24

1440

In [15]:
def get_kline(interval, days_forward=365, start_year=2022, symbol='BTCUSD'):
    t1 = time()

    data = []
    # Начало отсчёта: 1 января start_year
    base = int(dt.datetime(start_year, 1, 1, 0, 0, 0, 0).timestamp() * 1000)

    # Определяем количество "кадров" (баров), которое безопасно получать за один запрос.
    safe_frames = 720  # для 1-минутного интервала – 720 баров (12 часов)
    # Вычисляем длительность одного кадра (в мс) исходя из переданного интервала (в минутах)
    frame_duration_ms = interval * 60 * 1000
    # Вычисляем общее окно запроса в мс, пропорционально интервалу
    window_ms = frame_duration_ms * safe_frames

    # Общий период выборки в миллисекундах: days_forward дней
    total_ms = (60 * 1000) * (60 * 24 * days_forward)

    if interval == 1440:
        interval = 'D'

    for i in tqdm.tqdm(range(base, base + total_ms, window_ms)):
        returns = session.get_kline(
            symbol=symbol,
            interval=interval,
            start=i,
            end=(i + window_ms) - 1,  # конец окна запроса
            limit=1000
        )
        
        data.extend(returns['result']['list'])

        # sleep(1)  # можно включить задержку, если API требует ограничения по запросам

    output_df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover'])

    # Преобразуем столбцы в числовой формат
    for col in output_df.columns:
        try:
            output_df[col] = output_df[col].astype(int)
        except ValueError:
            output_df[col] = output_df[col].astype(float)

    # Преобразуем метки времени в удобный формат
    output_df['timestamp'] = output_df['timestamp'].apply(
        lambda x: dt.datetime.fromtimestamp(x / 1000).strftime('%Y-%m-%d %H:%M:%S')
    )
    output_df['timestamp'] = pd.to_datetime(output_df['timestamp'])
    output_df.sort_values(by='timestamp', inplace=True)

    print(f'Done in {int(time() - t1)} seconds.')
    return output_df


# df = get_kline(1)
df = get_kline(interval=1440, days_forward=(365*4))


100%|██████████| 3/3 [00:01<00:00,  2.62it/s]

Done in 1 seconds.


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1189 entries, 719 to 720
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  1189 non-null   datetime64[ns]
 1   open       1189 non-null   float64       
 2   high       1189 non-null   float64       
 3   low        1189 non-null   float64       
 4   close      1189 non-null   float64       
 5   volume     1189 non-null   int64         
 6   turnover   1189 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 74.3 KB


In [17]:
df['timestamp'].max() - df['timestamp'].min()

Timedelta('1188 days 00:00:00')

In [18]:
df['timestamp'].min()

Timestamp('2022-01-01 03:00:00')

In [19]:
df['timestamp'].max()

Timestamp('2025-04-03 03:00:00')

In [22]:
def check_time_series(df=df, interval=5):
    df['diff'] = df['timestamp'].diff()

    # Проверяем, что все разницы равны 1 минуте
    if (df['diff'].iloc[1:] != pd.Timedelta(minutes=interval)).any():
        print("Временной ряд не является непрерывным")
    else:
        print("Временной ряд непрерывен")

    df.drop(columns=['diff'], inplace=True)

check_time_series(interval=1440)

Временной ряд непрерывен


In [24]:
df.reset_index(drop=True, inplace=True)

In [25]:
df.to_parquet('btcusd_1d_4years_2022.parquet')

In [26]:
df2 = pd.read_parquet('btcusd_1d_4years_2022.parquet')

In [27]:
df2

,timestamp,open,high,low,close,volume,turnover
0,2022-01-01 03:00:00,46195.0,47915.0,46195.0,47724.0,955391852,20265.396710
1,2022-01-02 03:00:00,47724.0,47988.0,46656.0,47300.5,949091748,20087.631782
2,2022-01-03 03:00:00,47300.5,47614.5,45665.0,46436.0,1270992169,27240.417783
3,2022-01-04 03:00:00,46436.0,47500.0,45500.0,45829.5,1543295957,33235.051992
4,2022-01-05 03:00:00,45829.5,47058.0,42406.5,43409.0,2999423512,66521.756847
...,...,...,...,...,...,...,...
1184,2025-03-30 03:00:00,82581.5,83472.0,81427.1,82338.9,277774531,3360.382439
1185,2025-03-31 03:00:00,82338.9,83890.0,81234.0,82500.1,380843886,4615.113851
1186,2025-04-01 03:00:00,82500.1,85561.8,82376.7,85124.1,424790267,5051.613589
1187,2025-04-02 03:00:00,85124.1,88544.2,82252.2,82459.3,890695708,10410.198590


In [ ]:
# base = int(dt.datetime(2022, 1, 1, 0, 00, 00, 000000).timestamp() * 1000)

# dt.datetime.fromtimestamp(base / 1000).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# base + 60 * 1000
# dt.datetime.fromtimestamp((base + 60 * 1000) / 1000).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# l = []

# for i in range(base, base + (60 * 1000) * ((60*24)*(365*1)), (60 * 1000)*(12*60)):
#     # l.append(i)
#     print(dt.datetime.fromtimestamp(i / 1000).strftime('%Y-%m-%d %H:%M:%S'))
#     print(dt.datetime.fromtimestamp(((i + (60*(60*1000))*12)-1) / 1000).strftime('%Y-%m-%d %H:%M:%S'))
#     # print('')